In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

lib_path = Path('../../novelinsights/backend')
sys.path.append(str(lib_path))

In [2]:
from google import genai
import anthropic

from dotenv import load_dotenv
import os

load_dotenv(lib_path / '.env')
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "")

from novelinsights.services.ai.llmclient import AnthropicClient, GoogleGeminiClient

G_CLIENT = GoogleGeminiClient(genai.Client(api_key=GEMINI_API_KEY))
A_CLIENT = AnthropicClient(anthropic.Anthropic(api_key=ANTHROPIC_API_KEY))

USE_CACHED_RESPONSES = True

In [3]:
import json
from dataclasses import asdict

from novelinsights.utils.mocks.mock_story import load_story

STORY = load_story(file_dir=Path("../../novelinsights/backend/tests/resources/pokemon_amber/"))

CHAPTER_INDEX = 0

In [4]:
from novelinsights.services.ai.llmclient import LLMResponse
from novelinsights.utils.mocks.mock_llm import load_mock_summarize, load_mock_summarize_prompt, save_mock_summarize

summarize_prompt = load_mock_summarize_prompt(STORY, CHAPTER_INDEX)

summarize_resp: LLMResponse | None = None # type: ignore

if USE_CACHED_RESPONSES:
    summarize_resp = load_mock_summarize(STORY, CHAPTER_INDEX)

if not summarize_resp:
    summarize_resp = summarize_prompt.generate(client=A_CLIENT, store_config=True)
    save_mock_summarize(STORY, CHAPTER_INDEX, summarize_resp)

In [34]:
from novelinsights.utils.mocks.mock_llm import load_mock_find_entities, load_mock_find_entities_prompt, save_mock_find_entities
from novelinsights.schemas.prompt_responses.narrative.chapterbychapter.find_entities import FindEntitiesOutputSchema

find_entities_prompt = load_mock_find_entities_prompt(STORY, CHAPTER_INDEX)

find_entities_resp: LLMResponse[FindEntitiesOutputSchema] | None = None # type: ignore

if USE_CACHED_RESPONSES:
    find_entities_resp = load_mock_find_entities(STORY, CHAPTER_INDEX)

if not find_entities_resp:
    find_entities_resp = find_entities_prompt.generate_structured(client=G_CLIENT, store_config=True)
    save_mock_find_entities(STORY, CHAPTER_INDEX, find_entities_resp)

find_entities_resp: LLMResponse[FindEntitiesOutputSchema]

In [35]:
from novelinsights.services.ai.agents.narrative.chapterbychapter import KeyEntities
from novelinsights.types.knowledge import SignificanceLevel

MIN_SIG_LEVEL=SignificanceLevel.SUPPORTING

key_entities = KeyEntities(find_entities_resp.resp.entities)

print("-" * 100)
for i, upsert_chunk in enumerate(key_entities.yield_for_upsert(min_sig_level=MIN_SIG_LEVEL)):
    
    for upsert_str in upsert_chunk:
        print(upsert_str)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
CHARACTER: AlexaTheGreat/Amber - CENTRAL
	The main character, a Pokemon fan who isekais into the body of Amber, Dr. Fuji's deceased daughter.
	related: Dr. Fuji, Amber, Mewtwo, Gyarados
CHARACTER: Dr. Fuji - CENTRAL
	A scientist who is the father of Amber and involved in cloning and genetic experiments, specifically with Mewtwo. He shows great emotion and attachment towards the protagonist in Amber's body.
	related: AlexaTheGreat/Amber, Amber, Mewtwo
CHARACTER: Amber - MAJOR
	Dr. Fuji's deceased daughter, whose body the protagonist now inhabits. She was seemingly part of a cloning experiment.
	related: Dr. Fuji, AlexaTheGreat/Amber
CHARACTER: Mewtwo - MAJOR
	A genetically engineered Pokemon of immense power, created by Dr. Fuji. It escapes containment and causes destruction.
	related: Dr. Fuji, AlexaTheGreat/Amber
-------------------------------------------------------------------------

In [44]:
from novelinsights.schemas.prompt_responses.narrative.chapterbychapter.upsert_entities import UpsertEntitiesOutputSchema
from novelinsights.utils.mocks.mock_llm import load_mock_upsert_entities, load_mock_upsert_entities_prompt, save_mock_upsert_entities

upsert_entities_prompt = load_mock_upsert_entities_prompt(STORY, CHAPTER_INDEX)

upsert_entities_response_list: list[LLMResponse[UpsertEntitiesOutputSchema]] = []

if USE_CACHED_RESPONSES:
    upsert_entities_response_list = load_mock_upsert_entities(STORY, CHAPTER_INDEX)

if not upsert_entities_response_list:
    for i, upsert_chunk in enumerate(key_entities.yield_for_upsert(min_sig_level=MIN_SIG_LEVEL)):   
        upsert_entities_prompt.update_prompt_template(new_entities=upsert_chunk)
        
        upsert_entities_response = upsert_entities_prompt.generate_structured(client=G_CLIENT)
        save_mock_upsert_entities(STORY, CHAPTER_INDEX, upsert_entities_response, i)
        
        upsert_entities_response_list.append(upsert_entities_response)

In [41]:
from novelinsights.schemas.prompt_responses.narrative.chapterbychapter.upsert_entities import UpsertEntity, UpsertRelationship


all_entities: list[UpsertEntity] = []
all_relationships: list[UpsertRelationship] = []
for i in range(len(upsert_entities_response_list)):
    all_entities.extend(upsert_entities_response_list[i].resp.entities)
    all_relationships.extend(upsert_entities_response_list[i].resp.relationships)
    

In [42]:
for relationship in all_relationships:
    print(relationship)

relationship_type=<RelationType.SOCIAL: 'SOCIAL'> relationship_composition=<RelationCompositionType.ASSOCIATES: 'ASSOCIATES'> source_entity='Dr. Fuji' target_entity='AlexaTheGreat/Amber' significance_level=<SignificanceLevel.CENTRAL: 'CENTRAL'> strength=<RelationshipStrength.STRONG: 'STRONG'> current_description='Dr. Fuji is the creator and father figure to both Amber and Mewtwo. He is protective of Amber and seems to care for her deeply.' description='Dr. Fuji is the creator of Mewtwo and the father of Amber. He shows a strong emotional bond with Amber and wants to recreate his family. He is responsible for the events unfolding in the lab due to his experiments.'
relationship_type=<RelationType.SOCIAL: 'SOCIAL'> relationship_composition=<RelationCompositionType.BUILDS_UPON: 'BUILDS_UPON'> source_entity='AlexaTheGreat/Amber' target_entity='Dr. Fuji' significance_level=<SignificanceLevel.CENTRAL: 'CENTRAL'> strength=<RelationshipStrength.STRONG: 'STRONG'> current_description="Amber (Ale

In [43]:
for relationship in all_relationships:
    print(set({relationship.source_entity, relationship.target_entity}))

{'Dr. Fuji', 'AlexaTheGreat/Amber'}
{'Dr. Fuji', 'AlexaTheGreat/Amber'}
{'Mewtwo', 'Dr. Fuji'}
{'Dr. Fuji', 'Mewtwo'}
{'Mewtwo', 'AlexaTheGreat/Amber'}
{'Mewtwo', 'AlexaTheGreat/Amber'}
{'Gyarados', 'AlexaTheGreat'}
{'Laboratory', 'Dr. Fuji'}
{'Laboratory', 'Mewtwo'}
{'Amber', 'Laboratory'}
